**PACKAGES**

In [5]:
import pandas as pd
import numpy as np
import copy
import math
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [7]:
dataset = pd.read_csv("profil_ibu.csv")
dataset

,Usia (tahun),Tinggi Badan (m),Berat Badan (kg),Klasifikasi,IMT (kg/m2),Lama Kehamilan (hari),Berat Lahir Bayi (kg),Status
0,27.0,1.5748,45.359200,Normal,18.290037,284.0,3.401940,0
1,33.0,1.6256,61.234920,Gemuk,23.172440,282.0,3.203493,0
2,28.0,1.6256,52.163080,Normal,19.739486,279.0,3.628736,0
3,36.0,1.7526,86.182480,Obesitas,28.057784,0.0,3.486988,0
4,23.0,1.7018,56.699000,Normal,19.577551,282.0,3.061746,0
...,...,...,...,...,...,...,...,...
1231,27.0,1.5240,45.359200,Normal,19.529695,275.0,3.203493,0
1232,24.0,1.7018,54.431040,Normal,18.794449,265.0,3.628736,0
1233,30.0,1.6510,68.038800,Gemuk,24.961030,291.0,3.685435,0
1234,21.0,1.6510,49.895120,Normal,18.304755,281.0,3.543687,0


In [8]:
dataset.shape

(1236, 8)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Usia (tahun)           1236 non-null   float64
 1   Tinggi Badan (m)       1236 non-null   float64
 2   Berat Badan (kg)       1236 non-null   float64
 3   Klasifikasi            1236 non-null   object 
 4   IMT (kg/m2)            1217 non-null   float64
 5   Lama Kehamilan (hari)  1236 non-null   float64
 6   Berat Lahir Bayi (kg)  1236 non-null   float64
 7   Status                 1236 non-null   int64  
dtypes: float64(6), int64(1), object(1)
memory usage: 77.4+ KB


In [10]:
dataset.describe()

,Usia (tahun),Tinggi Badan (m),Berat Badan (kg),IMT (kg/m2),Lama Kehamilan (hari),Berat Lahir Bayi (kg),Status
count,1236.000000,1236.000000,1236.000000,1217.000000,1236.000000,1236.000000,1236.000000
mean,27.211165,1.597857,56.644319,inf,276.400485,3.389944,0.018608
std,5.879759,0.224438,13.571602,NaN,32.663624,0.516994,0.135192
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.559222,0.000000
25%,23.000000,1.574800,50.802304,19.911134,272.000000,3.083008,0.000000
50%,26.000000,1.625600,56.699000,21.256810,280.000000,3.401940,0.000000
75%,31.000000,1.676400,62.595696,23.344088,288.000000,3.713785,0.000000
max,45.000000,1.828800,113.398000,inf,353.000000,4.989512,1.000000


In [11]:
dataset = dataset.drop(['Klasifikasi','IMT (kg/m2)','Lama Kehamilan (hari)','Berat Lahir Bayi (kg)'], axis=1)

In [12]:
dataset.corr()

,Usia (tahun),Tinggi Badan (m),Berat Badan (kg),Status
Usia (tahun),1.000000,0.037873,0.126405,-0.049768
Tinggi Badan (m),0.037873,1.000000,0.552981,-0.822800
Berat Badan (kg),0.126405,0.552981,1.000000,-0.520107
Status,-0.049768,-0.822800,-0.520107,1.000000


In [13]:
#Output as labels
Y = dataset['Status'].values.reshape(dataset.shape[0],)
dataset = dataset.drop('Status',axis=1)
data = dataset.values
#Input as features
X = data[:,0:data.shape[1]]

#Splitting the Data into Training (90%) and Testing (10%)
X_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [14]:
print("First five elements in X_train are:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First five elements in X_train are:
 [[24.        1.6002   51.255896]
 [23.        1.6256   58.96696 ]
 [26.        1.6764   61.23492 ]
 [26.        1.7018   61.23492 ]
 [34.        1.6764   54.43104 ]]
Type of X_train: <class 'numpy.ndarray'>


In [15]:
print("First five elements in y_train are:\n", y_train[:5])
print("Type of y_train:",type(y_train))

First five elements in y_train are:
 [0 0 0 0 0]
Type of y_train: <class 'numpy.ndarray'>


In [16]:
#Check the dimensions of variables
print(f'The shape of X_train is: {str(X_train.shape)}')
print(f'The shape of y_train is: {str(y_train.shape)}')
print(f'We have m = {len(y_train)} training examples')

The shape of X_train is: (1112, 3)
The shape of y_train is: (1112,)
We have m = 1112 training examples


**LOGISTIC REGRESSION MODEL**

In [17]:
# Membentuk fungsi aktivasi sigmoid untuk membuat model
def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

In [18]:
# Mendefinisikan fungsi cost untuk evaluasi
def compute_cost(X, y, w, b, *argv):

    m, n = X.shape
    
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
             
    total_cost = cost / m

    return total_cost

In [19]:
def compute_gradient(X, y, w, b, *argv): 
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    for i in range(m):
        z_wb = 0
        for j in range(n): 
            z_wb += X[i, j] * w[j]
        z_wb += b
        f_wb = sigmoid(z_wb)
        
        dj_db_i = f_wb - y[i]
        dj_db += dj_db_i
        
        for j in range(n):
            dj_dw[j] += (f_wb - y[i]) * X[i][j]
            
    dj_dw = dj_dw/m
    dj_db = dj_db/m
        
    return dj_db, dj_dw

In [21]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    
    m = len(X)
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        if i<100000:      
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history

In [22]:
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(3) - 0.5)
initial_b = 10

iterations = 1000
alpha = 0.001

w,b, J_history,_ = gradient_descent(X_train ,y_train, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0)

Iteration    0: Cost     5.61   
Iteration  100: Cost     0.09   
Iteration  200: Cost     0.08   
Iteration  300: Cost     0.07   
Iteration  400: Cost     0.06   
Iteration  500: Cost     0.05   
Iteration  600: Cost     0.05   
Iteration  700: Cost     0.05   
Iteration  800: Cost     0.04   
Iteration  900: Cost     0.04   
Iteration  999: Cost     0.04   


In [24]:
def predict(X, w, b): 
  
    m, n = X.shape   
    p = np.zeros(m)
   
    for i in range(m):   
        z_wb = 0
        for j in range(n): 
            z_wb += X[i, j] * w[j]
        
        z_wb += b
        
        f_wb = sigmoid(z_wb)

        p[i] = f_wb >= 0.5
    return p

In [25]:
# Tes model prediksi
np.random.seed(1)
tmp_w = np.random.randn(3)
tmp_b = 0.3    
tmp_X = np.array([[20., 1.5, 36]]) - 0.5

tmp_p = predict(tmp_X, tmp_w, tmp_b)
print(f'Output of predict: shape {tmp_p.shape}, value {tmp_p}')

Output of predict: shape (1,), value [1.]


In [26]:
# Menghitung akurasi model prediksi
p = predict(X_test, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_test) * 100))

Train Accuracy: 98.381295


In [27]:
import pickle

with open("predict.pkl", "wb") as file:
  pickle.dump(predict, file)

In [28]:
import pickle

with open("predict.pkl", "rb") as file:
  model = pickle.load(file)

In [29]:
import tensorflow as tf


In [30]:
# Menentukan input layer
input_layer = tf.keras.Input(shape=(3,))  # Ubah 3 sesuai dengan jumlah fitur input Anda

# Menambahkan layer Dense (fully connected layer) dengan aktivasi sigmoid
dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')(input_layer)

# Menggabungkan input layer dan dense layer menjadi model
model_tf = tf.keras.Model(inputs=input_layer, outputs=dense_layer)


In [31]:
# Mengonversi model TensorFlow menjadi model TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp5j5snoo0\assets


In [ ]:
# Menyimpan model TFLite ke file
with open("model.tflite", "wb") as file:
  file.write(tflite_model)
